In [1]:
import dask.array as da
from dask.distributed import Client
from dask_ml.preprocessing import StandardScaler
from dask_ml.cluster import KMeans
import numpy as np
from dask_ml.metrics import pairwise_distances
from dask_ml.metrics import pairwise_distances_argmin_min
from time import time
from timeit import default_timer as timer
from dask_ml.datasets import make_blobs
import pandas as pd
from dask.distributed import SSHCluster
import matplotlib.pyplot as plt

In [2]:
def phi(X,C):
    return pairwise_distances(X, C, metric='sqeuclidean').min(1).sum() 
    # Compute the sum of distances to the nearest centroid at axis=1
    
def kmeans_parallel_init_dask(X, k, l):
    # Step 1: Randomly select a point from X
    n, d = X.shape
    idx = np.random.choice(n, size=1)
    C = X[idx].compute()  # Collect to memory for use

    # Step 2: Compute φX(C)
    phi_X_C = phi(X, C).compute() # Compute the sum of distances to the nearest centroid

    # Steps 3-6: Repeat O(log φX(C)) times
    rounds = int(np.log(phi_X_C))
    #print(f"Begin centroid sampling with number of rounds: {rounds}")
    for _ in range(rounds):
        dist_sq = pairwise_distances(X, C, metric='sqeuclidean').min(1)
        dist_sum = dist_sq.sum()
        p_x = l * dist_sq / dist_sum
        samples = da.random.uniform(size=len(p_x), chunks=p_x.chunks) < p_x
        sampled_idx = da.where(samples)[0].compute()
        
        new_C = X[sorted(sampled_idx)].compute() #https://github.com/dask/dask-ml/issues/39 
        C = np.vstack((C, new_C))

    # Step 7: Compute weights
    dist_to_C = pairwise_distances(X, C, metric='euclidean')
    closest_C = da.argmin(dist_to_C, axis=1)

    weights = np.empty(len(C))
    counts = da.bincount(closest_C, minlength=len(C)).compute()
    weights[:len(counts)] = counts
    
    # Normalize weights so that they sum up to the number of centroids
    weight_sum = np.sum(weights)
    if weight_sum == 0:
        raise ValueError("Sum of weights is zero, cannot normalize.")
    
    weights_normalized = weights / weight_sum
    
    dask_C = da.from_array(C, chunks=(C.shape[0], C.shape[1])) # here we ensure that the re-clustering occurs on a single-thread

    # Step 8: Recluster the weighted points in C into k clusters
    #print("Begin centroid re-clustering")
    labels, centroids = lloyd_kmeans_plusplus(X=dask_C, weights=weights_normalized,k=k, max_iters=10, tol=1e-8)

    return centroids

In [3]:
def update_centroids_weighted(X, labels, weights, k):
    """
    Update the centroids by computing the weighted mean of the points assigned to each cluster.
    """
    centroids = []
    
    for i in range(k):
        # Select the points that are assigned to cluster i
        cluster_points = X[labels == i]
        
        # Select the corresponding weights for these points
        cluster_weights = weights[labels == i]
        
        if len(cluster_points) == 0:
            # If no points are assigned to this cluster, avoid division by zero
            # Continue without updating that centroid
            continue

        # Compute the weighted mean using dask.array.average
        weighted_mean = da.average(cluster_points, axis=0, weights=cluster_weights)
        
        # Append the computed weighted mean to the centroids list
        centroids.append(weighted_mean)
    
    # Convert centroids list to Dask array
    return da.stack(centroids)

def kmeans_plusplus_init(X, weights, k):
    '''
    K-means++ initialization to select k initial centroids from X as a numpy array, keeping C as a NumPy array and weighting by provided weights.
    '''
    n, d = X.shape
    # Step 1: Randomly select the first centroid
    idx = np.random.choice(n, size=1)
    C = X[idx].compute()

    for _ in range(1, k):
        # Step 2: Compute distances from each point to the nearest centroid
        # C is a NumPy array, X is a Dask array
        # Compute the distances from each point to the nearest centroid normalizing by weights
        distances = pairwise_distances(X, C, metric='sqeuclidean').min(1) * (weights)
        
        # Compute the probabilities for choosing each point
        probabilities = distances / distances.sum()
        
        # Sample a new point based on these probabilities
        new_idx = np.random.choice(n, size=1, p=probabilities)
        new_centroid = X[sorted(new_idx)].compute()
        
        # Add the new centroid to the list
        C = np.vstack((C, new_centroid))
    return C

def lloyd_kmeans_plusplus(X, weights, k, max_iters=100, tol=1e-8):
    """
    Lloyd's algorithm for k-means clustering using Dask weighting the mean to update centroids.
    """
    centroids = kmeans_plusplus_init(X, weights, k)

    for i in range(max_iters):
        labels = assign_clusters(X, centroids).compute()
        new_centroids = update_centroids_weighted(X, labels, weights=weights, k=k).compute()
        
        if da.allclose(centroids, new_centroids, atol=tol).compute():
            #print(f"Centroid Lloyd Converged after {i+1} iterations.")
            break
        
        centroids = new_centroids

    return labels, centroids

In [4]:
def assign_clusters(X, centroids):
    """
    Assign each point to the nearest centroid using Dask to parallelize the computation.
    """
    return pairwise_distances_argmin_min(X, centroids, metric='sqeuclidean')[0]


def update_centroids(X, labels, k):
    """
    Update the centroids by computing the mean of the points assigned to each cluster with Dask.
    """
    centroids = da.stack([X[labels == i].mean(axis=0) for i in range(k)])
    return centroids
    
def kmeans_parallel(X, k, max_iters=100, tol=1e-8, l=2):
    centroids = kmeans_parallel_init_dask(X, k, l)
    for i in range(max_iters):
        labels = assign_clusters(X, centroids)
        new_centroids = update_centroids(X, labels, k).compute()

        if da.allclose(centroids, new_centroids, atol=tol):
            #print(f"Main KMeans Converged after {i+1} iterations.")
            break

        centroids = new_centroids

    return labels, centroids

In [5]:
# Load the KDDCUP99 dataset in a pd dataframe
file_path = '/opt/kddcup99/kddcup.data.gz'
df = pd.read_csv(file_path, compression='gzip', header=None)

# We identified those as the non-numerical columns
columns_to_drop = [1, 2, 3, 41]

# We remove those columns to accomodate all or part of the dataset into an array
df = df.drop(df.columns[columns_to_drop], axis=1)
num_rows = len(df)

num_sampled = int(num_rows * 1) #we can change the percentage of the dataset to sample

# And take only up to the desired part
df= df.iloc[:num_sampled]

# We initialize a numpy array ready to convert to a dask array during benchmark
data = df.values

del df

In [6]:
# Define the benchmarking function with multiple runs
def benchmark_kmeans(data, n_workers, threads_per_worker, memory_limit, chunks, n_runs):


    # List of IP addresses for the virtual machines (VMs) where Dask workers will run
    VM_IPS = ['10.67.22.89','10.67.22.89', '10.67.22.241', '10.67.22.171']
    #VM_IPS = ['10.67.22.89', '10.67.22.241', '10.67.22.171']

    # Initialize an SSHCluster with the specified VMs
    cluster = SSHCluster(
        hosts=VM_IPS,  # IP addresses of the VMs
        connect_options={
            "username": "tramarin"  # Username used to SSH into the VMs
        },
        worker_options={
            "n_workers": n_workers,  # Number of worker processes to launch per VM
            "nthreads":  threads_per_worker ,   # Number of threads per worker process
            "memory_limit": memory_limit
        },
        scheduler_options={
            "port": 8786,  # Port on the scheduler VM for communication with workers
            "dashboard_address": 8787,  # Port on the scheduler VM for the Dask dashboard
        },
    )

    client = Client(cluster)

    print(f"\nBenchmarking with {n_workers} workers, {threads_per_worker} threads per worker, "
          f"memory limit {memory_limit}, chunks {chunks}")

    # Step 2: import ddk as dask array

    dask_array = da.from_array(data, chunks=(data.shape[0] // chunks, data.shape[1]))

    scaler = StandardScaler(with_mean=True) # for scaling with a sparse matrix
    # we normalize the data to fasten clustering
    data_normalized = scaler.fit_transform(dask_array)
    del dask_array
    # persist the data to avoid re-computation in the benchmark
    data_normalized = data_normalized.persist()

    centers = 4

    # Step 3: Benchmark custom KMeans function
    custom_kmeans_times = []
    for _ in range(n_runs):
        start_time = timer()
        synt_labels, synt_centroids = kmeans_parallel(X=data_normalized, k=centers, l=2)
        custom_kmeans_times.append(timer() - start_time)
    
    custom_kmeans_mean_time = np.mean(custom_kmeans_times)
    custom_kmeans_std_time = np.std(custom_kmeans_times)

    # Step 4: Benchmark Dask-ML KMeans
    dask_ml_times = []
    for _ in range(n_runs):
        start_time = timer()
        dask_ml_kmeans = KMeans(n_clusters=centers).fit(data_normalized)
        dask_ml_times.append(timer() - start_time)

    dask_ml_mean_time = np.mean(dask_ml_times)
    dask_ml_std_time = np.std(dask_ml_times)


    # Record results
    results = {
        'workers': n_workers,
        'threads': threads_per_worker,
        'memory_limit': memory_limit,
        'chunks': chunks,
        'custom_kmeans_mean_time': custom_kmeans_mean_time,
        'custom_kmeans_std_time': custom_kmeans_std_time,
        'dask_ml_mean_time': dask_ml_mean_time,
        'dask_ml_std_time': dask_ml_std_time,
       
    }

    print(f"Custom KMeans Mean Time: {custom_kmeans_mean_time:.2f} seconds, Std: {custom_kmeans_std_time:.2f} seconds")
    print(f"Dask-ML KMeans Mean Time: {dask_ml_mean_time:.2f} seconds, Std: {dask_ml_std_time:.2f} seconds")
    
    
    client.close()
    cluster.close()
    
    return results

In [7]:
# Define the configurations to test
worker_configs = [
    {'n_workers': 1, 'threads_per_worker': 4, 'memory_limit': '7 GB'}, 
    {'n_workers': 3, 'threads_per_worker': 1, 'memory_limit': '2.6GB'},
    {'n_workers': 2, 'threads_per_worker': 2 , 'memory_limit': '3.2 GB'},
    {'n_workers': 1, 'threads_per_worker': 3 , 'memory_limit': '7 GB'},
    {'n_workers': 1, 'threads_per_worker': 2 , 'memory_limit': '7 GB'},
]

chunk_sizes = [
    12 
]

# Run benchmark for each configuration and chunk size
all_results = []
for config in worker_configs:
    for chunks in chunk_sizes:
        result = benchmark_kmeans(
            data=data,
            n_workers=config['n_workers'],
            threads_per_worker=config['threads_per_worker'],
            memory_limit=config['memory_limit'],
            chunks=chunks,
            n_runs=14 
        )
        all_results.append(result)


2024-09-01 17:47:35,724 - distributed.deploy.ssh - INFO - 2024-09-01 17:47:35,724 - distributed.scheduler - INFO - State start
2024-09-01 17:47:35,729 - distributed.deploy.ssh - INFO - 2024-09-01 17:47:35,728 - distributed.scheduler - INFO -   Scheduler at:    tcp://10.67.22.89:8786
2024-09-01 17:47:36,828 - distributed.deploy.ssh - INFO - 2024-09-01 17:47:36,827 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.89:41109'
2024-09-01 17:47:37,105 - distributed.deploy.ssh - INFO - 2024-09-01 17:47:37,116 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.171:35669'
2024-09-01 17:47:37,141 - distributed.deploy.ssh - INFO - 2024-09-01 17:47:37,141 - distributed.worker - INFO -       Start worker at:    tcp://10.67.22.89:37113
2024-09-01 17:47:37,155 - distributed.deploy.ssh - INFO - 2024-09-01 17:47:37,163 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.241:46589'
2024-09-01 17:47:37,411 - distributed.deploy.ssh - INFO - 2024-09-01 


Benchmarking with 1 workers, 4 threads per worker, memory limit 7 GB, chunks 12


/opt/daskenv1/lib/python3.12/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 1.11 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/opt/daskenv1/lib/python3.12/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 1.11 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/opt/daskenv1/lib/python3.12/site-packages/dask/base.py:1539: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  w

Custom KMeans Mean Time: 8.61 seconds, Std: 2.09 seconds
Dask-ML KMeans Mean Time: 6.92 seconds, Std: 0.32 seconds


2024-09-01 17:51:27,239 - distributed.deploy.ssh - INFO - 2024-09-01 17:51:27,238 - distributed.scheduler - INFO - State start
2024-09-01 17:51:27,240 - distributed.deploy.ssh - INFO - 2024-09-01 17:51:27,239 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-scratch-space/worker-pdhndhtw', purging
2024-09-01 17:51:27,242 - distributed.deploy.ssh - INFO - 2024-09-01 17:51:27,242 - distributed.scheduler - INFO -   Scheduler at:    tcp://10.67.22.89:8786
2024-09-01 17:51:28,349 - distributed.deploy.ssh - INFO - 2024-09-01 17:51:28,349 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.89:34777'
2024-09-01 17:51:28,352 - distributed.deploy.ssh - INFO - 2024-09-01 17:51:28,352 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.89:44721'
2024-09-01 17:51:28,354 - distributed.deploy.ssh - INFO - 2024-09-01 17:51:28,354 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.89:41297'
2024-09-01 17:51:28,565 - distr

...

Custom KMeans Mean Time: 10.97 seconds, Std: 3.44 seconds
Dask-ML KMeans Mean Time: 6.93 seconds, Std: 0.49 seconds


2024-09-01 18:04:29,426 - distributed.deploy.ssh - INFO - 2024-09-01 18:04:29,426 - distributed.scheduler - INFO - State start
2024-09-01 18:04:29,427 - distributed.deploy.ssh - INFO - 2024-09-01 18:04:29,426 - distributed.diskutils - INFO - Found stale lock file and directory '/tmp/dask-scratch-space/worker-nx7v7xl9', purging
2024-09-01 18:04:29,430 - distributed.deploy.ssh - INFO - 2024-09-01 18:04:29,429 - distributed.scheduler - INFO -   Scheduler at:    tcp://10.67.22.89:8786
2024-09-01 18:04:30,538 - distributed.deploy.ssh - INFO - 2024-09-01 18:04:30,538 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.89:35681'
2024-09-01 18:04:30,752 - distributed.deploy.ssh - INFO - 2024-09-01 18:04:30,753 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.171:44291'
2024-09-01 18:04:30,777 - distributed.deploy.ssh - INFO - 2024-09-01 18:04:30,778 - distributed.nanny - INFO -         Start Nanny at: 'tcp://10.67.22.241:34197'
2024-09-01 18:04:30,854 - dis


Benchmarking with 1 workers, 2 threads per worker, memory limit 7 GB, chunks 12


/opt/daskenv1/lib/python3.12/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 1.11 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/opt/daskenv1/lib/python3.12/site-packages/distributed/client.py:3362: UserWarning: Sending large graph of size 1.11 GiB.
This may cause some slowdown.
Consider loading the data with Dask directly
 or using futures or delayed objects to embed the data into the graph without repetition.
See also https://docs.dask.org/en/stable/best-practices.html#load-data-with-dask for more information.
  warnings.warn(
/opt/daskenv1/lib/python3.12/site-packages/dask/base.py:1539: UserWarning: Running on a single-machine scheduler when a distributed client is active might lead to unexpected results.
  w

Custom KMeans Mean Time: 11.11 seconds, Std: 3.37 seconds
Dask-ML KMeans Mean Time: 7.54 seconds, Std: 0.42 seconds


In [9]:
# Convert results to DataFrame
results_df = pd.DataFrame(all_results)

# Save results to a file in Parquet format
results_df.to_pickle('benchmark_results_cluster_config_80PERCDATA_14runs.pickle')